In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import recall_score

In [2]:
npz = np.load('/Users/shahzadiaiman/.cache/kagglehub/datasets/saurabhbagchi/ship-and-iceberg-images/versions/1/input_data.npz')

X_train = npz['X_train']
Y_train = npz['Y_train']
del npz

print('We have {} examples to work with'.format(Y_train.shape[0]))

We have 4113 examples to work with


In [3]:
# Normalize
X_train = X_train.astype(np.float32)
X_train = (X_train - X_train.mean()) / X_train.std()

# Split into train/validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

In [4]:
model = Sequential([
    Flatten(input_shape=X_train.shape[1:]),   # flatten image -> vector

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='sigmoid')  # binary classification: ship vs iceberg
])

/Users/shahzadiaiman/Library/Python/3.9/lib/python/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [6]:
# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64
)

Epoch 1/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.6559 - loss: 0.7572 - val_accuracy: 0.7060 - val_loss: 0.7304
Epoch 2/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.9229 - loss: 0.2412 - val_accuracy: 0.7521 - val_loss: 0.6234
Epoch 3/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.9731 - loss: 0.0996 - val_accuracy: 0.7922 - val_loss: 0.6383
Epoch 4/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.9824 - loss: 0.0617 - val_accuracy: 0.7886 - val_loss: 0.6726
Epoch 5/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.9935 - loss: 0.0299 - val_accuracy: 0.7801 - val_loss: 0.7348
Epoch 6/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.9930 - loss: 0.0252 - val_accuracy: 0.7388 - val_loss: 0.8953
Epoch 7/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.9937 - loss: 0.0221 - val_accuracy: 0.7874 - val_loss: 0.7746
Epoch 8/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.9968 - loss: 0.0177 - val_accuracy: 0.7704 - v

In [9]:
# Evaluate
loss, acc = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {acc:.4f}")

# Get predictions
y_pred_probs = model.predict(X_val)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Compute recall
recall = recall_score(y_val, y_pred)
print(f"Validation Recall: {recall:.4f}")

Validation Accuracy: 0.7679
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Validation Recall: 0.7234
